<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [2]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [3]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [4]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f''' 
select
    count(*) as N_row,
    count(distinct id) as N_ids,
    count(distinct name) as N_names
from
    vacancies
limit 10
'''

In [6]:
vacancies_df = pd.read_sql_query(query_3_1, connection)
vacancies_df

,n_row,n_ids,n_names
0,49197,49197,21223


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
query_3_2 = '''
select
    count(distinct name) as N_employers,
    count(distinct id) as N_ids,
    count(*) as N_rows
from
    employers
'''

In [8]:
employers_df = pd.read_sql_query(query_3_2, connection)
employers_df

,n_employers,n_ids,n_rows
0,23175,23501,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
query_3_3 = ''' 
select
    count(distinct name) as N_areas,
    count(distinct id) as N_ids
from
    areas   
'''

In [10]:
areas_df = pd.read_sql_query(query_3_3, connection)
areas_df

,n_areas,n_ids
0,1362,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
query_3_4 =''' 
select
    count(distinct name) as N_areas,
    count(distinct id) as N_ids
from
    industries       
'''

In [12]:
idustries_df = pd.read_sql_query(query_3_4, connection)
idustries_df

,n_areas,n_ids
0,294,294


***

1. В таблице vacancies, которая отражает общее количество вакансий, 49197 строк. <br>
Кличество уникальных названий вакансий составило 21223
2. В таблице employers, которая отражает общее количество работодателей, 23501 строка. <br>
Кличество уникальных названий работадателя составило 23175
3. В таблице areas, которая отражает общее количество регионов, 1362 строки. <br>
Все названия (names) уникальны.

4. В таблице industries которая отражает количество сфер деятельности, 294 строки <br>
Все названия (names) уникальны.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
query_4_1 = ''' 
select
    a.name,
    count(*) N_vacs
from
    vacancies v
    join areas a
    on v.area_id = a.id
group by a.name
order by count(*) desc
limit 10
'''

In [14]:
VacAreaJoin = pd.read_sql_query(query_4_1, connection)
VacAreaJoin

,name,n_vacs
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
query_4_2 = ''' 
select
    count(*) as N_slry_non_null
from 
    vacancies
where 
    (salary_from is not null) or (salary_to is not null)
'''


In [16]:
slry_not_null = pd.read_sql_query(query_4_2, connection)
slry_not_null

,n_slry_non_null
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
query_4_3 = ''' 
select
    round(avg(salary_from)) as avg_slry_from,
    round(avg(salary_to)) as avg_slry_to
from 
    vacancies

'''

In [18]:
avg_slry = pd.read_sql_query(query_4_3, connection)
avg_slry

,avg_slry_from,avg_slry_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. <br> Результат отсортируйте по убыванию количества.


In [19]:
query_4_4 = ''' 
select
    schedule,
    employment,
    count(*) N_vac
from
    vacancies
group by 
    schedule, employment
order by count(*) desc
'''

In [20]:
sch_empl_count = pd.read_sql_query(query_4_4, connection)
display(sum(sch_empl_count.loc[2:, 'n_vac']))
sch_empl_count

6028

,schedule,employment,n_vac
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
query_4_5 = ''' 
select
    experience,
    count(*) as N_vac
from
    vacancies
group by experience
order by count(*) asc
'''

In [22]:
count_by_exp = pd.read_sql_query(query_4_5, connection)
count_by_exp

,experience,n_vac
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

1. Наибольшее количество вакансий представлено в Москве и Санкт-Петербурге. Также, в топе столицы сосдних государств и города-миллионники.

2. Только у 24073 соискателей из 49197 (чуть менее половины) указана хотя бы одна граница зарплатной вилки

3. В среднем, нижняя граница зарплатной вилки составила 71к рублей, верхней - 110к рублей

4. Соискатели, преимущественно, предпочитают полную занятость и полный день (~35к). На втором месте удаленная работа и полная занятость (~8к).<br>
На все остальные сочетания графика работы и занятости приходится ~6к
5. Подавляющее бльшинство соискателей (~26к) имеет опыть работы до 3-х лет. От 3-х до 6-ти лет у ~14к соискателей. <br>
~7к соискателей не имеют опыта работы. Опыт работы более 6 лет имеют всего 1337 соискателей.

ЗАМЕНИТЬ СОИСКАТЕЛЕЙ НА ВАКАНСИИ


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
query_5_1 = ''' 
select
    e.name,
    count(*) N_vacs
from
    vacancies v
    join employers e
    on v.employer_id = e.id
group by
    e.name  
order by count(*) desc
'''

In [24]:
vac_count_by_empl = pd.read_sql_query(query_5_1, connection)
vac_count_by_empl.iloc[:5, :]

,name,n_vacs
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём. <br>
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:
query_5_2 =''' 
select
    a.name,
    count(e.name) N_employers
    
from
    areas a
    left join vacancies v 
    on v.area_id = a.id
    
    join employers e 
    on a.id = e.area
    
where 
    v.id is null
    
group by 
    a.name

order by 
    count(e.name) desc
'''

In [26]:
Null_vac = pd.read_sql_query(query_5_2, connection)
Null_vac

,name,n_employers
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18
...,...,...
181,Миллерово,1
182,"Михайловка (Уфимский район, Башкортостан)",1
183,Можайск,1
184,Морозовск\n,1


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
query_5_3 = ''' 
select
    e.name as Employer_name,
    count(distinct a.name) as N_regions_with_vac
from
    vacancies v
    join employers e
    on v.employer_id = e.id
    
    join areas a
    on v.area_id = a.id
    
group by 
    e.name
    
order by 
    count(distinct a.name) desc
'''


In [28]:
N_region_by_Employer = pd.read_sql_query(query_5_3, connection)
N_region_by_Employer

,employer_name,n_regions_with_vac
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
query_5_4 = ''' 
select 
    count(e.name) N_empl_with_no_indstr
from
    employers e
    left join employers_industries ei 
    on ei.employer_id = e.id
where industry_id is null

'''


In [30]:
N_Empl_null_Indstr = pd.read_sql_query(query_5_4, connection)
N_Empl_null_Indstr

,n_empl_with_no_indstr
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
query_5_5 = ''' 
select 
    e.name,
    count(ei.industry_id)
    
from
    employers_industries ei
    join employers e
    on ei.employer_id = e.id 
    
group by 
    e.name

having 
    count(ei.industry_id) = 4
    
order by 
    e.name
    
offset 2
limit 1
'''

In [32]:
Indstr_by_Empl = pd.read_sql_query(query_5_5, connection)
Indstr_by_Empl

,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
query_5_6 = ''' 
select 
    i.name,
    count (*) N_emplyers
    
from
    employers_industries ei
    join employers e
    on ei.employer_id = e.id 
    
    join industries i
    on ei.industry_id = i.id

group by 
    i.name
    
having 
    i.name = 'Разработка программного обеспечения'
'''


In [34]:
N_dev_Empl = pd.read_sql_query(query_5_6, connection)
N_dev_Empl

,name,n_emplyers
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
import requests
from bs4 import BeautifulSoup

# Устанавливаем библиотеки BeautifulSoup и lxml

#!pip install beautifulsoup4 
#!pip install lxml

# Выделяем таблицу с названиями городов-миллионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_="standard sortable")


# Создаем список с названиями городов-миллионников
df = pd.read_html(str(table))
df = pd.concat(df) 
city_names = list(df.loc[:,'Город'])

In [36]:
query_5_7 = ''' 
select 
    a.name as City,
    v.id as vacancy_cnt
    
from 
    vacancies v
    join employers e
    on v.employer_id = e.id
    
    join areas a
    on v.area_id = a.id

where e.name = 'Яндекс' 
'''

In [37]:
# Записываем результат запроса в датафрейм
YaVac_by_City = pd.read_sql_query(query_5_7, connection)


# Выбираем только данные из городов-миллионников
mask = YaVac_by_City.loc[:, 'city'].apply(lambda x: x in city_names)
YaVac_by_City = YaVac_by_City[mask]


# Группируем по переменной 'City', подссчитываем кол-во вакансий
YaVac_by_City = \
    YaVac_by_City.groupby('city').count()\
        .sort_values('vacancy_cnt', ascending=False)


# Добавляем строрку 'Total' с суммой по солбцу с вакансиями
YaVac_by_City.loc['Total'] = sum(YaVac_by_City['vacancy_cnt'])
display(YaVac_by_City)


,vacancy_cnt
city,
Москва,54
Санкт-Петербург,42
Екатеринбург,39
Нижний Новгород,36
Новосибирск,35
Воронеж,32
Краснодар,30
Самара,26
Уфа,26


***

1.	Наибольшее количество вакансий (1933) предлагает компания **Яндекс**. В пятерку лидеров также входят **Ростелеком**, **Тинькофф**, **СБЕР** и **Газпром нефть**.

2.	Компании **Яндекс**, **Ростелеком** и **Спецремонт** входят в тройку лидеров по количеству регионов, в которых они предлагают вакансии. 

3.	Компания **Яндекс** предлагает **485** вакансий в городах-миллионниках. Наибольшее количество вакансий приходится на **Москву**, **Санкт-Петербург**, **Екатеринбург**, **Нижний Новгород** и **Новосибирск**.

4.	Среди всех работодателей, у **8419** не указан род деятельности. У **3553** компаний в качестве рода деятельности указана разработка программного обеспеченья.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [38]:
query_6_1 = ''' 
select
    name
from
    vacancies
where 
    lower(name) like '%data%' 
    or 
    lower(name) like '%данн%'
'''

Data_Vac = pd.read_sql_query(query_6_1, connection)
Data_Vac

,name
0,Аналитик хранилищ данных и BI-систем/Аналитик ...
1,Data-analysts (удаленно)
2,Стажер Data Engineer/Data Аналитик в команду Б...
3,Старший специалист хранилищ данных
4,Администратор баз данных и сетей
...,...
1766,Программист Delphi / разработчик баз данных
1767,Разработчик баз данных (Big Data)
1768,Аналитик данных
1769,Data аналитик /Аналитик BI (удаленно)


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
query_6_2 = ''' 
select
    name,
    experience,
    employment
    
from
    vacancies
    
where 
    (lower(name) like 'ml%'or
     lower(name) like '% ml%'or
     lower(name) like '%-ml%'or
     lower(name) like '%/ml%'or
     lower(name) like '%(ml%'or
     
        
     lower(name) like '%data scientist%' or 
     lower(name) like '%data science%' or 
     lower(name) like '%исследователь данных%' or
     lower(name) like '%machine learning%' or 
     lower(name) like '%машинн%обучен%')
     
    and
    
    (lower(name) like '%junior%' or 
     employment like 'Cтажировка' or
     experience like 'Нет опыта')
'''


In [40]:
Data_Vac_Jun = pd.read_sql_query(query_6_2, connection)
Data_Vac_Jun

,name,experience,employment
0,Junior+ CV/ML developer,От 1 года до 3 лет,Полная занятость
1,Data Scientist (Junior+),От 1 года до 3 лет,Полная занятость
2,Junior Data Scientist (Стажер),Нет опыта,Стажировка
3,"Математик, Junior/Middle/Senior Data Scientist",От 1 года до 3 лет,Частичная занятость
4,Data Scientist/Machine Learning Engineer (Deve...,Нет опыта,Частичная занятость
5,Junior Data Scientist,Нет опыта,Полная занятость
6,Data Scientist (Junior),Нет опыта,Частичная занятость
7,Intern/Junior Data Scientist,Нет опыта,Стажировка
8,Junior Data Scientist (VK),От 1 года до 3 лет,Полная занятость
9,Junior/Middle Data Scientist,От 1 года до 3 лет,Полная занятость


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
query_6_3 = ''' 
select 
    id,
    name,
    key_skills

from 
    vacancies

where 
    ((name ilike '%ML%' and name not ilike '%HTML%') or
        
    (lower(name) like '%data scientist%') or 
    (lower(name) like '%data science%') or
    (lower(name) like '%исследователь данных%') or
    (lower(name) like '%machine learning%') or
    (lower(name) like '%машинн%обучен%') and 
    
    (key_skills IS NOT NULL))

    AND
    
    (key_skills like '%SQL%' or key_skills ilike '%postgres%')

order by name
    
'''


In [42]:
Data_Vac_SQL = pd.read_sql_query(query_6_3, connection)
Data_Vac_SQL

,id,name,key_skills
0,54593096,Analyst/Data Scientist,Английский язык\tНавыки презентации\tСбор и ан...
1,54686859,B2B Data Scientist (Эксперт по моделированию и...,Python\tSQL\tPostgreSQL\tGit\tАнглийский язык
2,51834014,Backend- разработчик (Middle/Senior-TeamLeader),JavaScript\tMySQL\tОбучение и развитие\tУправл...
3,54759229,Chief Data Science (Цифровой кредитный монитор...,Python\tSpark\tHadoop\tSQL\tBig Data
4,55048457,Commercial Data Scientist / Эксперт по наукоем...,Python\tData Analysis\tMachine Learning\tTenso...
...,...,...,...
224,54671675,Специалист по разработке систем технического з...,Python\tLinux\tMongoDB\tPostgreSQL
225,54820149,Специалист по углубленной аналитике данных (Da...,SPSS\tСтатистический анализ\tStatistica\tАнали...
226,54924841,Стажер направления Машинное обучение и анализ ...,SQL\tOracle Pl/SQL
227,55156731,Стажёр Data Scientist / Группа сервисов Антифр...,Python\tSQL\tPostgreSQL\tML\tMachine Learning\...


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
query_6_4 = ''' 
select 
    id,
    name,
    key_skills

from 
    vacancies

where 
    ((name ilike '%ML%' and name not ilike '%HTML%') or
        
    (lower(name) like '%data scientist%') or 
    (lower(name) like '%data science%') or
    (lower(name) like '%исследователь данных%') or
    (lower(name) like '%machine learning%') or
    (lower(name) like '%машинн%обучен%') )
    
    AND 
    
    (key_skills IS NOT NULL)

    AND
    
    (lower(key_skills) like '%python%')

order by name
    
'''

In [44]:
Data_Vac_Python = pd.read_sql_query(query_6_4, connection)
Data_Vac_Python

,id,name,key_skills
0,66120847,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit
1,66120842,AI/ML Senior Engineer,Python\tFlask\tLinux\tAWS\tGit
2,54593096,Analyst/Data Scientist,Английский язык\tНавыки презентации\tСбор и ан...
3,54686859,B2B Data Scientist (Эксперт по моделированию и...,Python\tSQL\tPostgreSQL\tGit\tАнглийский язык
4,54759229,Chief Data Science (Цифровой кредитный монитор...,Python\tSpark\tHadoop\tSQL\tBig Data
...,...,...,...
352,54820149,Специалист по углубленной аналитике данных (Da...,SPSS\tСтатистический анализ\tStatistica\tАнали...
353,55156731,Стажёр Data Scientist / Группа сервисов Антифр...,Python\tSQL\tPostgreSQL\tML\tMachine Learning\...
354,55033440,Старший разработчик (ML engineer),Python\tLinux\tPyTorch\tMachine Learning\tData...
355,55453238,Эксперт по машинному обучению и искусственному...,Python\tМатематическая статистика\tМатематичес...


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
query_6_5 = ''' 
select 
    v.name,
    v.key_skills,
    length(v.key_skills) - length(replace(v.key_skills, CHR(9), '')) + 1  as n_skills

from 
    vacancies v

where 
   
    (
    (name ILIKE '%data scientist%') 
    OR (name ILIKE '%data science%') 
    OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%')  
    )
    
    AND (key_skills IS NOT NULL)
    
    order by v.name
'''


In [46]:
DS_Vacs = pd.read_sql_query(query_6_5, connection)
Avg_vacs = round(DS_Vacs['n_skills'].mean(), 2)

print(f'Среднее количество запрашиваемых навыков составляет: {Avg_vacs}')

Среднее количество запрашиваемых навыков составляет: 6.55


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [186]:
query_6_6 = ''' 
select 
    name,
    coalesce(salary_from, 0) salary_from,
    coalesce(salary_to, 0) salary_to,   
    experience
    
from 
    vacancies v

where 
    
    (salary_from is not null or salary_to is not null)
    AND
    
    ((name ILIKE '%data scientist%') 
    OR (name ILIKE '%data science%') 
    OR (name ILIKE '%исследователь данных%')
    OR ((name ILIKE '%ML%') AND (name NOT ILIKE '%HTML%'))
    OR (name ILIKE '%machine learning%')
    OR (name ILIKE '%машинн%обучен%'))  
            
order by v.name
'''

DS_Avg_Slry = pd.read_sql_query(query_6_6, connection)
DS_Avg_Slry

,name,salary_from,salary_to,experience
0,Backend- разработчик (Middle/Senior-TeamLeader),160000,200000,От 1 года до 3 лет
1,C++ Machine Learning Engineer (Computer Vision),0,233794,От 3 до 6 лет
2,C++ Machine Learning Engineer (Computer Vision),0,233794,От 3 до 6 лет
3,C++ Machine Learning Engineer (Computer Vision),0,233794,От 3 до 6 лет
4,C++ разработчик (QML),60000,0,От 1 года до 3 лет
...,...,...,...,...
79,Руководитель направления Data Science,0,300000,От 3 до 6 лет
80,Руководитель отдела разработки программного об...,200000,0,Более 6 лет
81,Системный аналитик /Middle+/Senior/TeamLead,252000,319000,От 3 до 6 лет
82,Специалист Data Science (R),100000,120000,От 1 года до 3 лет


In [ ]:
# создаем функцию Avg_slry для поиска средней ЗП
def Avg_slry(data_frame):
    
    """ 
        Эта функция расчитывает среднее по ЗП, если оба 
        значения не равны нулю. Если значение salary_from == 0, 
        принимает значение salary_to. И наоборот, Если значение 
        salary_to == 0, принимает значение salary_from.

    Args:
        data_frame (pd.dataframe): Датафрейм, в котором есть колонки 
        salary_from (ЗП от) и salary_to (ЗП до)

    Returns:
        pd.dataframe: Датафрейм с дополнительным столбцом 'Avg_salary' 
        (среднее значение по ЗП)
    """
    
    for i in range (0, len(data_frame)):
    
        if data_frame.loc[i, 'salary_from'] == 0 or \
            data_frame.loc[i, 'salary_to'] == 0:
                
            data_frame.loc[i, 'Avg_salary'] = \
                (data_frame.loc[i, 'salary_to'] + \
                    data_frame.loc[i, 'salary_from'])
            
               
        else: data_frame.loc[i, 'Avg_salary'] = \
            (data_frame.loc[i, 'salary_to'] + data_frame.loc[i, 'salary_from']) / 2
            
    return data_frame 

In [ ]:
# применяем функцию Avg_slry к датафрейму
DS_Avg_Slry = Avg_slry(DS_Avg_Slry)
display(DS_Avg_Slry.head())


# группируем значения по переменной 'experience' и определяем среднее
group_slry = round(DS_Avg_Slry.groupby(['experience'], as_index=False)
                   .mean(['Avg_salary']).sort_values(['Avg_salary']))

group_slry.loc[:, ['experience', 'Avg_salary']]


,name,salary_from,salary_to,experience,Avg_salary
0,Backend- разработчик (Middle/Senior-TeamLeader),160000,200000,От 1 года до 3 лет,180000.0
1,C++ Machine Learning Engineer (Computer Vision),0,233794,От 3 до 6 лет,233794.0
2,C++ Machine Learning Engineer (Computer Vision),0,233794,От 3 до 6 лет,233794.0
3,C++ Machine Learning Engineer (Computer Vision),0,233794,От 3 до 6 лет,233794.0
4,C++ разработчик (QML),60000,0,От 1 года до 3 лет,60000.0


,experience,Avg_salary
1,Нет опыта,74643.0
2,От 1 года до 3 лет,144233.0
0,Более 6 лет,157933.0
3,От 3 до 6 лет,256454.0


***

In [49]:
# выводы по предметному анализу

# Общий вывод по проекту

In [50]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования